### Data Testing
After training and saving the model to local, now it is time to test it! First apply the same data reformatting and processing method on the task dataset, then feed the processed data to model to calculate accuracy.

In [2]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
import os

from random import shuffle, seed

In [3]:
index_dict = {'Pulling_OneH': 0, 'Overhead': 1, 'Pulling': 2, 'Sitting': 3, 
              'Lifting': 4, 'Crawling': 5, 'Standing': 6, 'Carrying': 7, 
              'Walking': 8, 'Pushing': 9, 'Reaching': 10, 'Static_Stoop': 11, 
              'Kneeling': 12, 'Lifting_OneH': 13, 'Crouching': 14}

header = ['TimeSec', 'Sensor', 'Quatx', 'Quaty', 'Quatz', 'Quat0', 'Heading',
       'Pitch', 'Roll', 'LinAccx', 'LinAccy', 'LinAccz', 'Vbat', 'Accx',
       'Accy', 'Accz', 'Gyrox', 'Gyroy', 'Gyroz']

# actually {1.0, 3.0, 5.0, 6.0, 8.0, 10.0, 12.0, 14.0} in the task data

In [ ]:
directory = "./Spinetrack Data/task/"
name_dir = []

for d in sorted(os.listdir(directory)):
    name_dir.append(d)
#name_dir

total_data = []
for d in name_dir:
    try:
        # parsing timestep txt file
        index_text = open('./Spinetrack Data/task/{0}/timeStamps_Everything.txt'.format(d), 'r').readlines()
        act_index = []
        for line in index_text[1:]:
            row = line.rstrip().split('\t')
            if len(row[2].split('_')) > 1 and row[2].split('_')[1] == 'OneH':
                row[2] = '_'.join(row[2].split('_')[:2])
            else:
                row[2] = row[2].split('_')[0]

            if row[2] not in index_dict.keys():
                row.append(999)
            else:
                row.append(index_dict[row[2]])

            act_index.append(row)
        index_df = pd.DataFrame(act_index)
        index_df.columns = ['Frame', 'Time(s)', 'Task', 'Index']

        # parsing everything csv file
        recording_df = pd.read_csv('./Spinetrack Data/task/{0}/Everything.csv'.format(d), error_bad_lines=False).dropna()
        recording_df.columns = header
#         print(recording_df)

        # adding activity columns
        frame = []
        for index in range(1, len(index_df)):
            activity_index = index_df.iloc[index - 1]['Index']
            if activity_index != 999:
                time_lower_bound = float(index_df.iloc[index - 1]['Time(s)'])
                time_upper_bound = float(index_df.iloc[index]['Time(s)'])
                df_part = recording_df.loc[(recording_df['TimeSec'] >= (time_lower_bound + 2)) & (recording_df['TimeSec'] < (time_upper_bound - 2))]
                df_part['activity'] = [int(activity_index) for i in range(len(df_part))]
                frame.append(df_part)
        data_df = pd.concat(frame)
        total_data.append(data_df)
    except:
        pass

result_df = pd.concat(total_data)
# result_df

### Hyperparameter
window_size: The number of timesteps in one window (e.g. how many rows in one window).

channel: The number of features in one window. Similar to image channels (RGB).

batch_size: The numebr of windows in one batch.

learning_rate: How fast the model learns.

In [4]:
window_size = 540
channel = 1
batch_size = 32
learning_rate = 0.001

---
### Go to 
<a href=#bookmark> Run all cell above</a>

## Data Reformatting

In [28]:
ranking = {}
for window_size in range(10, 750, 10):
#     window_size = i
    
    
    # Process original dataset, create windows (window_size samples(rows), about 1 second)
    data = []
    window = 1
    while window*window_size < len(result_df):
        data_window = result_df[(window - 1)*window_size:window*window_size]
        data.append(data_window.values)
        window += 1
    #data

    # delete window if multiple activities and sensors presents
    cleaned_data = []
    for i in data:
        previous_activity = -1
        previous_sensor = -1
        for j in i:
            current_activity = j[19]
            current_sensor = j[1]
            if (previous_activity != -1) and (current_activity != previous_activity):
#                 print("data contains different activities! Window droped")
                break
    #         elif (previous_sensor != -1) and (current_sensor != previous_sensor):
    #             print("data contains different sensors! Window droped")
    #             break
            else:
                previous_activity = current_activity
                previous_sensor = current_sensor
        else:
            cleaned_data.append(i)

    # extract label from each window
    labels = []
    for i in cleaned_data:
        label = i[0][19]
        labels.append(label)
    labels = np.array(labels)
    #labels

    # extract features from each window
    features = []
    for i in cleaned_data:
        new = np.delete(i, 19, 1)
        features.append(new)
    features = np.array(features)
    #features

    # combine the features and labels
    k = list(zip(features, labels))
    activity_data = pd.DataFrame(k)
    activity_data.columns = ['features', 'labels']
    activity_data

    # check the size of activity. The final output of neural net 
    # has to have max_index + 1 output
    max_index = activity_data['labels'].max()
    label_size = int(max_index + 1)

    # define our dataset in pytorch
    class DatasetSpineTrack(Dataset):

        def __init__(self, file, transform=None):
            #self.data = pd.read_csv(file_path)
            self.data = file
            self.transform = transform

        def __len__(self):
            return len(self.data)

        def __getitem__(self, index):
            # load image as ndarray type (Height * Width * Channels)
            # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
            # in this example, i don't use ToTensor() method of torchvision.transforms
            # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)

            features = torch.tensor(self.data["features"].iloc[index])
            features = features.view(channel, window_size, 19) 
            labels = torch.tensor(self.data["labels"].iloc[index], dtype=torch.long)
            #print(labels.type())

    #         if self.transform is not None:
    #             image = self.transform(image)

            return features, labels

    # create test data set
    activity_data_test = activity_data
    test_dataset = DatasetSpineTrack(activity_data_test)

    # load data
    testloader = DataLoader(test_dataset, batch_size=batch_size)

    train_on_gpu = False

    # define the model
    model = models.resnet18(pretrained=False)
    # window_size channels
    model.conv1 = torch.nn.Conv2d(channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = torch.nn.Linear(512, label_size, bias=True)
    model.add_module("dropout", torch.nn.Dropout(p=0.5))
    model = model.double()

    # move tensors to GPU is CUDA is available
    if train_on_gpu:
        model.cuda()
#     print(model)

    # load model
    # model.load_state_dict(torch.load('model_Spinetrack_3.pt'))

    # if wanty to run in cpu...
    model.load_state_dict(torch.load('model_Spinetrack_10p.pt', map_location=lambda storage, loc: storage))

    # track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(len(index_dict)))
    class_total = list(0. for i in range(len(index_dict)))

    criterion = nn.CrossEntropyLoss()
    #criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.eval()
    torch.no_grad()
    # iterate over test data
    for features, labels in testloader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(features)
        # calculate the batch loss
        loss = criterion(output, labels)
        # update test loss 
        test_loss += loss.item()*features.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(labels.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

        # calculate test accuracy for each object class
        for i in range(batch_size):
            try:
                label = labels.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
            except:
                pass

    # average test loss
    test_loss = test_loss/len(testloader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

#     for i in range(len(index_dict)):
#         if class_total[i] > 0:
#             print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
#                 list(index_dict.keys())[i], 100 * class_correct[i] / class_total[i],
#                 np.sum(class_correct[i]), np.sum(class_total[i])))
#         else:
#             print('Test Accuracy of %5s: N/A (no training examples)' % (list(index_dict.keys())[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d) for (%2d) window_size' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total), window_size))
    print()
    
    accuracy = 100. * np.sum(class_correct) / np.sum(class_total)
    ranking[window_size] = accuracy
    
print(sorted(ranking.items(), key=lambda x: x[1], reverse=True))

Test Loss: 3.675390


Test Accuracy (Overall): 34% (2426/7057) for (10) window_size

Test Loss: 2.822246


Test Accuracy (Overall): 47% (1658/3492) for (20) window_size

Test Loss: 2.615161


Test Accuracy (Overall): 49% (1132/2304) for (30) window_size

Test Loss: 2.555013


Test Accuracy (Overall): 58% (999/1711) for (40) window_size

Test Loss: 2.593030


Test Accuracy (Overall): 59% (804/1353) for (50) window_size

Test Loss: 2.480481


Test Accuracy (Overall): 59% (668/1117) for (60) window_size

Test Loss: 2.482020


Test Accuracy (Overall): 60% (568/946) for (70) window_size

Test Loss: 2.627272


Test Accuracy (Overall): 65% (536/821) for (80) window_size

Test Loss: 2.568760


Test Accuracy (Overall): 62% (453/720) for (90) window_size

Test Loss: 2.359683


Test Accuracy (Overall): 62% (398/640) for (100) window_size

Test Loss: 2.596349


Test Accuracy (Overall): 66% (384/578) for (110) window_size

Test Loss: 2.455124


Test Accuracy (Overall): 65% (341/523) for (120) windo

---
## model 3 (60):
(740, 86.11111111111111), (660, 80.43478260869566), (720, 80.0), (520, 76.71232876712328), (710, 76.3157894736842), (580, 76.27118644067797), (540, 76.11940298507463), (650, 76.08695652173913), (640, 76.0), (700, 75.60975609756098), (560, 75.0), (630, 75.0), (680, 73.80952380952381), (460, 73.33333333333333), (620, 73.07692307692308), (480, 72.61904761904762), (600, 72.22222222222223), (490, 72.15189873417721), (690, 72.09302325581395), (510, 72.0), (550, 71.875), (570, 71.1864406779661), (450, 70.96774193548387), (500, 70.88607594936708), (400, 70.27027027027027), (380, 70.24793388429752), (350, 70.14925373134328), (590, 69.49152542372882), (340, 69.28571428571429), (280, 69.1891891891892), (370, 68.8), (530, 68.57142857142857), (440, 68.42105263157895), (730, 68.42105263157895), (410, 68.18181818181819), (360, 67.6923076923077), (670, 67.44186046511628), (260, 67.3170731707317), (300, 67.26190476190476), (470, 67.05882352941177), (420, 66.99029126213593), (320, 66.45161290322581), (240, 66.07929515418502), (310, 66.04938271604938), (430, 66.0), (220, 65.74803149606299), (390, 65.51724137931035), (270, 64.43298969072166), (290, 64.20454545454545), (330, 64.1891891891892), (610, 64.15094339622641), (180, 63.69230769230769), (250, 63.25581395348837), (200, 63.066202090592334), (190, 62.622950819672134), (160, 62.234042553191486), (210, 62.0817843866171), (120, 61.95028680688336), (230, 61.666666666666664), (170, 61.03151862464183), (150, 60.89108910891089), (140, 60.869565217391305), (100, 59.6875), (130, 59.62343096234309), (110, 59.16955017301038), (90, 58.333333333333336), (80, 57.97807551766139), (70, 57.399577167019025), (60, 57.38585496866607), (50, 55.432372505543235), (40, 49.4447691408533), (30, 42.751736111111114), (20, 38.373424971363114), (10, 14.453733881252656)

---
## model 5 (120):
[(730, 78.94736842105263), (620, 75.0), (670, 72.09302325581395), (520, 71.23287671232876), (700, 70.73170731707317), (640, 70.0), (720, 70.0), (650, 69.56521739130434), (660, 69.56521739130434), (740, 69.44444444444444), (630, 69.23076923076923), (600, 68.51851851851852), (490, 68.35443037974683), (510, 68.0), (480, 67.85714285714286), (580, 67.79661016949153), (560, 67.1875), (540, 67.16417910447761), (530, 67.14285714285714), (680, 66.66666666666667), (320, 66.45161290322581), (250, 66.04651162790698), (500, 65.82278481012658), (400, 65.76576576576576), (550, 65.625), (690, 65.11627906976744), (350, 64.92537313432835), (310, 64.81481481481481), (460, 64.44444444444444), (340, 64.28571428571429), (420, 64.07766990291262), (440, 63.1578947368421), (360, 63.07692307692308), (260, 62.4390243902439), (610, 62.264150943396224), (280, 62.16216216216216), (200, 62.02090592334495), (410, 61.81818181818182), (220, 61.811023622047244), (210, 61.71003717472119), (240, 61.67400881057269), (370, 61.6), (450, 61.29032258064516), (160, 61.170212765957444), (380, 61.15702479338843), (590, 61.016949152542374), (120, 60.994263862332694), (230, 60.833333333333336), (300, 60.714285714285715), (180, 60.61538461538461), (710, 60.526315789473685), (390, 60.3448275862069), (190, 60.32786885245902), (270, 59.79381443298969), (570, 59.32203389830509), (140, 59.267734553775746), (150, 59.15841584158416), (430, 59.0), (470, 58.8235294117647), (330, 58.78378378378378), (170, 58.16618911174785), (290, 57.38636363636363), (90, 57.083333333333336), (80, 57.003654080389765), (60, 54.25246195165622), (110, 53.806228373702425), (100, 52.5), (130, 52.09205020920502), (50, 50.25868440502587), (70, 46.723044397463), (30, 33.767361111111114), (40, 31.09292811221508), (20, 31.013745704467354), (10, 8.289641490718436)]

---
## model 10 (45):
[(590, 77.96610169491525), (740, 77.77777777777777), (710, 76.3157894736842), (570, 76.27118644067797), (680, 76.19047619047619), (660, 76.08695652173913), (510, 74.66666666666667), (540, 74.6268656716418), (690, 74.4186046511628), (490, 73.41772151898734), (700, 73.17073170731707), (470, 72.94117647058823), (720, 72.5), (460, 72.22222222222223), (600, 72.22222222222223), (450, 72.04301075268818), (430, 72.0), (560, 71.875), (610, 71.69811320754717), (440, 71.57894736842105), (390, 71.55172413793103), (520, 71.23287671232876), (580, 71.1864406779661), (400, 71.17117117117117), (620, 71.15384615384616), (630, 71.15384615384616), (730, 71.05263157894737), (380, 70.24793388429752), (410, 70.0), (530, 70.0), (640, 70.0), (420, 69.90291262135922), (300, 69.64285714285714), (500, 69.62025316455696), (370, 69.6), (330, 69.5945945945946), (650, 69.56521739130434), (290, 69.31818181818181), (480, 69.04761904761905), (360, 68.46153846153847), (140, 68.19221967963387), (230, 67.91666666666667), (240, 67.84140969162996), (260, 67.8048780487805), (320, 67.74193548387096), (250, 67.44186046511628), (310, 67.28395061728395), (350, 67.16417910447761), (340, 67.14285714285714), (280, 67.02702702702703), (130, 66.94560669456067), (190, 66.55737704918033), (210, 66.54275092936803), (220, 66.53543307086615), (170, 66.189111747851), (120, 66.1567877629063), (270, 65.97938144329896), (150, 65.84158415841584), (110, 65.7439446366782), (80, 65.6516443361754), (180, 65.53846153846153), (160, 65.42553191489361), (90, 65.27777777777777), (670, 65.11627906976744), (200, 64.80836236933798), (100, 64.6875), (50, 64.44937176644494), (60, 64.27931960608774), (550, 64.0625), (70, 63.00211416490486), (40, 61.776738749269434), (30, 53.21180555555556), (20, 51.3745704467354), (10, 29.233385291200225)]

---
## model 11 (90):
[(740, 72.22222222222223), (690, 72.09302325581395), (730, 71.05263157894737), (700, 70.73170731707317), (660, 69.56521739130434), (550, 68.75), (630, 67.3076923076923), (510, 66.66666666666667), (410, 66.36363636363636), (350, 65.67164179104478), (720, 65.0), (370, 64.8), (490, 64.55696202531645), (570, 64.40677966101696), (590, 64.40677966101696), (520, 64.38356164383562), (610, 64.15094339622641), (560, 64.0625), (710, 63.1578947368421), (480, 63.095238095238095), (650, 63.04347826086956), (310, 62.96296296296296), (270, 62.88659793814433), (530, 62.857142857142854), (330, 62.83783783783784), (210, 62.825278810408925), (670, 62.7906976744186), (580, 62.71186440677966), (540, 62.6865671641791), (470, 62.35294117647059), (360, 62.30769230769231), (400, 62.16216216216216), (440, 62.10526315789474), (390, 62.06896551724138), (380, 61.98347107438016), (680, 61.904761904761905), (280, 61.62162162162162), (260, 61.46341463414634), (190, 61.31147540983606), (300, 61.30952380952381), (450, 61.29032258064516), (420, 61.16504854368932), (150, 61.13861386138614), (460, 61.111111111111114), (600, 61.111111111111114), (430, 61.0), (250, 60.93023255813954), (240, 60.79295154185022), (320, 60.645161290322584), (220, 60.62992125984252), (290, 60.22727272727273), (340, 60.0), (640, 60.0), (620, 59.61538461538461), (180, 59.38461538461539), (90, 58.47222222222222), (160, 58.244680851063826), (500, 58.22784810126582), (170, 58.16618911174785), (230, 57.5), (110, 57.09342560553633), (130, 56.69456066945607), (120, 56.40535372848949), (140, 56.2929061784897), (60, 54.789615040286485), (200, 54.35540069686411), (80, 52.862362971985384), (70, 52.00845665961945), (50, 51.6629711751663), (100, 51.09375), (30, 48.35069444444444), (40, 41.72998246639392), (20, 39.289805269186715), (10, 20.546974635114072)]

In [1005]:
# single test set


# index_text = open('./Spinetrack Data/task/Alex_task/timeStamps_Everything.txt', 'r').readlines()
# act_index = []
# for line in index_text[1:]:
#     row = line.rstrip().split('\t')
#     if len(row[2].split('_')) > 1 and row[2].split('_')[1] == 'OneH':
#         row[2] = '_'.join(row[2].split('_')[:2])
#     else:
#         row[2] = row[2].split('_')[0] 
    
#     if row[2] not in index_dict.keys():
#         row.append(999)
#     else:
#         row.append(index_dict[row[2]])
    
#     act_index.append(row)
# index_df = pd.DataFrame(act_index)
# index_df.columns = ['Frame', 'Time(s)', 'Task', 'Index']

# recording_df = pd.read_csv('./Spinetrack Data/task/Alex_task/Everything.csv', error_bad_lines=False).dropna()
# recording_df.columns = header
# print(recording_df)

# frame = []
# for index in range(1, len(index_df)):
#     activity_index = index_df.iloc[index - 1]['Index']
#     if activity_index != 999:
#         time_lower_bound = float(index_df.iloc[index - 1]['Time(s)'])
#         time_upper_bound = float(index_df.iloc[index]['Time(s)'])
#         df_part = recording_df.loc[(recording_df['TimeSec'] >= (time_lower_bound + 2)) & (recording_df['TimeSec'] < (time_upper_bound - 2))]
#         df_part['activity'] = activity_index
#         frame.append(df_part)

# data_df = pd.concat(frame)
# data_df

# result_df = data_df

## Load Data

In [1025]:
# Release all the GPU memory cache that can be freed
torch.cuda.empty_cache() 

### Overall accuracy and each class accuracy

In [1026]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(len(index_dict)))
class_total = list(0. for i in range(len(index_dict)))

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
initial
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.eval()
torch.no_grad()
# iterate over test data
for features, labels in testloader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        features, labels = features.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(features)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update test loss 
    test_loss += loss.item()*features.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

    # calculate test accuracy for each object class
    for i in range(batch_size):
        try:
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        except:
            pass

# average test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(index_dict)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            list(index_dict.keys())[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (list(index_dict.keys())[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.572834

Test Accuracy of Pulling_OneH: N/A (no training examples)
Test Accuracy of Overhead: 100% (13/13)
Test Accuracy of Pulling: N/A (no training examples)
Test Accuracy of Sitting: 66% ( 4/ 6)
Test Accuracy of Lifting: N/A (no training examples)
Test Accuracy of Crawling: 100% ( 5/ 5)
Test Accuracy of Standing: 46% ( 6/13)
Test Accuracy of Carrying: N/A (no training examples)
Test Accuracy of Walking: 71% ( 5/ 7)
Test Accuracy of Pushing: N/A (no training examples)
Test Accuracy of Reaching: 76% (20/26)
Test Accuracy of Static_Stoop: N/A (no training examples)
Test Accuracy of Kneeling: 100% ( 1/ 1)
Test Accuracy of Lifting_OneH: N/A (no training examples)
Test Accuracy of Crouching: 37% ( 3/ 8)

Test Accuracy (Overall): 72% (57/79)


---
### Run all above 
<a name='bookmark' />